In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
#Define image size and batch size
IMG_SIZE = 224
BATCH_SIZE = 32

In [ ]:
#Define data generators for train,validation and test sets
train_datagen=ImageDataGenerator(rescale=1./255,validation_split=0.2)
train_generator=train_datagen.flow_from_directory('/content/drive/MyDrive/bt/Brain_Tumor_Detection/train',
target_size=(IMG_SIZE,IMG_SIZE),
batch_size=BATCH_SIZE,
class_mode='binary',
subset='training'
)
val_generator = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/bt/Brain_Tumor_Detection/train',target_size=(IMG_SIZE,IMG_SIZE),
batch_size=BATCH_SIZE,
class_mode='binary',
subset='validation'
)
test_datagen=ImageDataGenerator(rescale=1./255)
test_generator=test_datagen.flow_from_directory('/content/drive/MyDrive/bt/Brain_Tumor_Detection/test',
target_size=(IMG_SIZE,IMG_SIZE),
batch_size=BATCH_SIZE,
class_mode='binary'
)

NameError: name 'ImageDataGenerator' is not defined

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#Define  the model
model = keras.Sequential([
    layers.Conv2D(32,(3,3),activation='relu',input_shape=(IMG_SIZE,IMG_SIZE,3)),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64,(3,3),activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64,(3,3),activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Flatten(),
    layers.Dense(128,activation='relu'),
    layers.Dense(1,activation='sigmoid')
])

In [ ]:
#Compile the model
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
history=model.fit(train_generator,validation_data=val_generator,epochs=5)

Epoch 1/5
75/75 [==============================] - 325s 4s/step - loss: 0.4401 - accuracy: 0.8054 - val_loss: 0.2887 - val_accuracy: 0.8783
Epoch 2/5
75/75 [==============================] - 246s 3s/step - loss: 0.2195 - accuracy: 0.9129 - val_loss: 0.1383 - val_accuracy: 0.9433
Epoch 3/5
75/75 [==============================] - 245s 3s/step - loss: 0.1179 - accuracy: 0.9621 - val_loss: 0.1463 - val_accuracy: 0.9317
Epoch 4/5
75/75 [==============================] - 243s 3s/step - loss: 0.0629 - accuracy: 0.9808 - val_loss: 0.0441 - val_accuracy: 0.9867
Epoch 5/5
75/75 [==============================] - 240s 3s/step - loss: 0.0461 - accuracy: 0.9887 - val_loss: 0.0356 - val_accuracy: 0.9883


In [ ]:
model.save("Model.h5","label.txt")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np

#Load the saved model
model=load_model('/content/Model.h5')

#Load and preprocess the text image
test_image_path='/content/drive/MyDrive/bt/Brain_Tumor_Detection/train/yes/y1.jpg'
img=image.load_img(test_image_path,target_size=(224,224))
img_array=image.img_to_array(img)
img_array=np.expand_dims(img_array,axis=0)   #Add batch dimensions
img_array/=255.    #Normalize pixel values
#Make predictions
prediction=model.predict(img_array)
#Print prediction
if prediction< 0.5 :
  print("Prediction : No tumor (Probability :",prediction[0][0],")")
else :
  print("Prediction : Tumor present(Probability : ",prediction[0][0],")")

1/1 [==============================] - 0s 112ms/step
Prediction : Tumor present(Probability :  0.9999983 )
